In [2]:
from __future__ import absolute_import,division,print_function
import tensorflow as tf
tf.enable_eager_execution()
import matplotlib.pyplot as plt
import numpy as np
import pandas
import os
import unicodedata
import time
import re

In [0]:
path_to_file='/content/drive/My Drive/workshop/Question answer2.txt'

In [0]:
def convert_unicode_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD',s) if(unicodedata.category(c)!='Mn'))

In [0]:
def preprocessing(w):
    w=convert_unicode_ascii(w.lower().strip())
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    w=re.sub(r'[^a-zA-Z0-9.,?!¿]+',' ',w)
    w=w.rstrip().strip()
    w='<start> '+ w +' <end>'
    return w

In [0]:
def create_dataset(path):
    lines=open(path,encoding='UTF-8').read().strip().split('\n')
#     print(lines)
    word_pairs=[[preprocessing(w) for w in l.split('\t')] for l in lines]
#     word_pairs=[[preprocessing(l)] for l in lines]
    return word_pairs

In [0]:
class LanguageIndex():
    def __init__(self,lang):
        self.lang=lang
        self.word2idx={}
        self.idx2word={}
        self.vocab=set()
        self.create_index()
    def create_index(self):
        for phrase in self.lang:
            self.vocab.update(phrase.split(" "))
        self.vocab=sorted(self.vocab)
        self.word2idx['<pad>']=0
        for index,word in enumerate(self.vocab):
#             print(word,index)
            self.word2idx[word]=index+1
        for word,index in self.word2idx.items():
            self.idx2word[index]=word

In [0]:
def max_length(tensor):
    return max(len(t) for t in tensor)
def load_dataset(path):
    pairs=create_dataset(path)
    inp_lang=LanguageIndex(ques for ques,ans in pairs)
    tar_lang=LanguageIndex(ans for ques,ans in pairs)
    input_tensor=[[inp_lang.word2idx[s] for s in ques.split(' ')] for ques,ans in pairs]
    target_tensor=[[tar_lang.word2idx[s] for s in ans.split(' ')] for ques,ans in pairs]
    max_length_inp,max_length_tar=max_length(input_tensor),max_length(target_tensor)
    input_tensor=tf.keras.preprocessing.sequence.pad_sequences(input_tensor,maxlen=max_length_inp,padding="post")
    target_tensor=tf.keras.preprocessing.sequence.pad_sequences(target_tensor,maxlen=max_length_tar,padding="post")
    return input_tensor,target_tensor,inp_lang,tar_lang,max_length_inp,max_length_tar

In [0]:
# num_examples=628
input_tensor,target_tensor,inp_lang,targ_lang,max_length_inp,max_length_targ=load_dataset(path_to_file)
# print(inp_lang.word2idx)

In [0]:
from sklearn.model_selection import train_test_split
input_tensor_train,input_tensor_test,target_tensor_train,target_tensor_test=train_test_split(input_tensor,target_tensor,test_size=0.2)

In [0]:
buffer_size=len(input_tensor_train)
batch_size=64
num_batches=buffer_size//batch_size
embedding_dim=256
units=1024
vocab_inp_size=len(inp_lang.word2idx)
vocab_tar_size=len(targ_lang.word2idx)
dataset=tf.data.Dataset.from_tensor_slices((input_tensor_train,target_tensor_train)).shuffle(buffer_size)
dataset=dataset.batch(batch_size,drop_remainder=True)

In [0]:
def gru(units):
    if(tf.test.is_gpu_available()):
        return tf.keras.layers.CuDNNGRU(units,return_sequences=True,return_state=True,recurrent_initializer="glorot_uniform")
    else:
        return tf.keras.layers.GRU(units,return_sequences=True,return_state=True,recurrent_activation='softmax',recurrent_initializer="glorot_uniform")

In [0]:
class Encoder(tf.keras.Model):
    def __init__(self,vocab_size,embedding_dim,enc_units,batch_size):
        super(Encoder,self).__init__()
        self.batch_size=batch_size
        self.enc_units=enc_units
        self.embedding=tf.keras.layers.Embedding(vocab_size,embedding_dim)
        self.gru=gru(self.enc_units)
    def call(self,x,hidden):
        x=self.embedding(x)
        output,state=self.gru(x,initial_state=hidden)
        return output,state
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_size,self.enc_units))

In [0]:
class Decoder(tf.keras.Model):
    def __init__(self,vocab_size,embedding_dim,dec_units,batch_size):
        super(Decoder,self).__init__()
        self.batch_size=batch_size
        self.dec_units=dec_units
        self.embedding=tf.keras.layers.Embedding(vocab_size,embedding_dim)
        self.gru=gru(self.dec_units)
        self.fc=tf.keras.layers.Dense(vocab_size)
        self.w1=tf.keras.layers.Dense(self.dec_units)
        self.w2=tf.keras.layers.Dense(self.dec_units)
        self.v=tf.keras.layers.Dense(1)
    def call(self,x,hidden,enc_output):
        hidden_with_time_axis=tf.expand_dims(hidden,1)
        score=self.v(tf.nn.tanh(self.w1(enc_output)+self.w2(hidden_with_time_axis)))
        attention_weight=tf.nn.softmax(score,axis=1)
        context_vector=attention_weight*enc_output
        context_vector=tf.reduce_sum(context_vector,axis=1)
        x=self.embedding(x)
        x=tf.concat([tf.expand_dims(context_vector,axis=1),x],axis=-1)
        output,state=self.gru(x)
        output=tf.reshape(output,(-1,output.shape[2]))
        x=self.fc(output)
        return x,state,attention_weight
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_size,self.dec_units))

In [0]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, batch_size)
decoder = Decoder(vocab_tar_size, embedding_dim, units, batch_size)

In [0]:
optimizer = tf.train.AdamOptimizer()


def loss_function(real, pred):
  mask = 1 - np.equal(real, 0)
  loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
  return tf.reduce_mean(loss_)

In [0]:
checkpoint_dir = 'training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [0]:
EPOCHS = 600

for epoch in range(EPOCHS):
    start = time.time()
    
    hidden = encoder.initialize_hidden_state()
    total_loss = 0
    
    for (batch, (inp, targ)) in enumerate(dataset):
        loss = 0
        
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(inp, hidden)
            
            dec_hidden = enc_hidden
            
            dec_input = tf.expand_dims([targ_lang.word2idx['<start>']] * batch_size, 1)       
            
            # Teacher forcing - feeding the target as the next input
            for t in range(1, targ.shape[1]):
                # passing enc_output to the decoder
                predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
                
                loss += loss_function(targ[:, t], predictions)
                
                # using teacher forcing
                dec_input = tf.expand_dims(targ[:, t], 1)
        
        batch_loss = (loss / int(targ.shape[1]))
        
        total_loss += batch_loss
        
        variables = encoder.variables + decoder.variables
        
        gradients = tape.gradient(loss, variables)
        
        optimizer.apply_gradients(zip(gradients, variables))
        
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)
    
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / num_batches))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 2.7996
Epoch 1 Loss 2.2759
Time taken for 1 epoch 15.43695330619812 sec

Epoch 2 Batch 0 Loss 2.0246
Epoch 2 Loss 1.8599
Time taken for 1 epoch 10.002718210220337 sec

Epoch 3 Batch 0 Loss 1.8672
Epoch 3 Loss 1.7592
Time taken for 1 epoch 9.40089225769043 sec

Epoch 4 Batch 0 Loss 1.7809
Epoch 4 Loss 1.6871
Time taken for 1 epoch 9.778929471969604 sec

Epoch 5 Batch 0 Loss 1.6998
Epoch 5 Loss 1.6054
Time taken for 1 epoch 9.397048950195312 sec

Epoch 6 Batch 0 Loss 1.6092
Epoch 6 Loss 1.5190
Time taken for 1 epoch 9.786042928695679 sec

Epoch 7 Batch 0 Loss 1.5229
Epoch 7 Loss 1.4418
Time taken for 1 epoch 9.507150650024414 sec

Epoch 8 Batch 0 Loss 1.4581
Epoch 8 Loss 1.3811
Time taken for 1 epoch 9.668091773986816 sec

Epoch 9 Batch 0 Loss 1.4000
Epoch 9 Loss 1.3255
Time taken for 1 epoch 9.496330738067627 sec

Epoch 10 Batch 0 Loss 1.3552
Epoch 10 Loss 1.2769
Time taken for 1 epoch 9.652904033660889 sec

Epoch 11 Batch 0 Loss 1.3084
Epoch 11 Loss 1.2361
Time tak

In [0]:
def evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    
    sentence = preprocessing(sentence)
    print(sentence)
    inputs = [inp_lang.word2idx[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    
    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word2idx['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        
        # storing the attention weigths to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.idx2word[predicted_id] + ' '

        if targ_lang.idx2word[predicted_id] == '<end>':
            return result, sentence, attention_plot
        
        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

In [0]:
def translate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    result, sentence, attention_plot = evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)
        
    print('Input: {}'.format(sentence))
    print('Predicted reply: {}'.format(result))
    
    attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
#     plot_attention(attention_plot, sentence.split(' '), result.split(' '))

In [0]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [0]:
translate(u"Shall we meet today?", encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

<start> shall we meet today ? <end>
Input: <start> shall we meet today ? <end>
Predicted reply: how about at night . <end> 


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
